In [ ]:
# import packages
import pandas as pd
import numpy as np
from datetime import datetime

import ast

In [ ]:
# read the datasets
orginal_data_path = 'original_data/'

movies_metadata = pd.read_csv(orginal_data_path + 'movies_metadata.csv')
ratings = pd.read_csv(orginal_data_path + 'ratings.csv')
links = pd.read_csv(orginal_data_path + 'links.csv')
keywords = pd.read_csv(orginal_data_path + 'keywords.csv')
credits = pd.read_csv(orginal_data_path + 'credits.csv')

ratings['date_time'] = pd.to_datetime(ratings['timestamp'], unit='s')

# movies_metadata

In [ ]:
def get_list_dict(original_list, name, type = 'list'):
    if type == 'list':
        try:
            final_list = []
            for item in ast.literal_eval(original_list):
                final_list.append(item[name])
            return final_list
        except:
            pass
    elif type == 'dict':
        try:
            return ast.literal_eval(original_list)[name]
        except:
            pass 

In [ ]:
movies_metadata['lang_code_list'] = movies_metadata['spoken_languages'].apply(lambda x: get_list_dict(x, 'iso_639_1'))

movies_metadata['pro_comp_list'] = movies_metadata['production_companies'].apply(lambda x: get_list_dict(x, 'name'))

movies_metadata['pro_coun_list'] = movies_metadata['production_countries'].apply(lambda x: get_list_dict(x, 'iso_3166_1'))

movies_metadata['genres_list'] = movies_metadata['genres'].apply(lambda x: get_list_dict(x, 'name'))

movies_metadata['collection'] = movies_metadata['belongs_to_collection'].apply(lambda x: get_list_dict(x, 'id', 'dict'))

In [ ]:
# print the count of languages
lang_count_dict = dict()
for lang_list in movies_metadata['lang_code']:
    try:
        for lang in lang_list:
            if lang not in lang_count_dict.keys():
                lang_count_dict[lang] = 1
            else:
                lang_count_dict[lang] += 1
    except:
        pass

print(f"Total unique languages: {len(lang_count_dict)}")
print(lang_count_dict)

In [ ]:
# print 100-time language
onehundred_time_langs_list = []

for lang in lang_count_dict.keys():
    if lang_count_dict[lang] <= 100:
        onehundred_time_langs_list.append(lang)

print(onehundred_time_langs_list)

Production Companies & Countries

In [ ]:
# print the count of production companies
pro_comp_count_dict = dict()
for pro_comp_list in movies_metadata['pro_coun']:
    try:
        for pro_comp in pro_comp_list:
            if pro_comp not in pro_comp_count_dict.keys():
                pro_comp_count_dict[pro_comp] = 1
            else:
                pro_comp_count_dict[pro_comp] += 1
    except:
        pass

print(f"Total unique languages: {len(pro_comp_count_dict)}")
print(pro_comp_count_dict)

Genre

In [ ]:
# clean the genres
movies_metadata['genres_list'] = movies_metadata['genres'].apply(ast.literal_eval)

In [ ]:
# print the count of genres
all_genres = dict()
for genre_list in movies_metadata['genres_list']:
    for genre in genre_list:
        if genre['name'] not in all_genres.keys():
            all_genres[genre['name']] = 1
        else:
            all_genres[genre['name']] += 1

print(f"Total unique genres: {len(all_genres)}")
print(all_genres)

In [ ]:
# print one-time genre
onetime_genres = []

for genre in all_genres.keys():
    if all_genres[genre] == 1:
        onetime_genres.append(genre)
  
print(onetime_genres)

In [ ]:
# get the dummy variables for genres
for genre in all_genres.keys():
    movies_metadata[f"genre_{genre}"] = movies_metadata['genres_list'].apply(
        lambda x: int(any(g['name'] == genre for g in x))
    )
   
# remove the one-time genre dummy variable
movies_metadata = movies_metadata.drop([f"genre_{genre}" for genre in onetime_genres], axis = 1)

Save csv file

In [ ]:
movies_metadata.to_csv('movies_cleaned.csv', index=False)